In [1]:
# read table data
from pygmid import Lookup as lk
import numpy as np
lv_nmos = lk('sg13_lv_nmos.mat')
lv_pmos = lk('sg13_lv_pmos.mat')
# list of parameters: VGS, VDS, VSB, L, W, NFING, ID, VT, GM, GMB, GDS, CGG, CGB, CGD, CGS, CDD, CSS, STH, SFL
# if not specified, minimum L, VDS=max(vgs)/2=0.9 and VSB=0 are used 

In [16]:
# define the given parameters as taken from the specification table or initial guesses
c_load = 1e-12
gm_id_m12 = 14
gm_id_m34 = 12
gm_id_m56 = 10
gm_id_m78 = 10
gm_id_m90 = 10
gm_id_mt = 7
l_12 = 2
l_34 = 2
l_56 = 2
l_78 = 2
l_90 = 2
l_t = 5
f_bw = 1e6 # -3dB bandwidth of the voltage buffer
i_total_limit = 20e-6 # we plan 2x5uA in addition for additional bias voltage generation
i_bias_in = 14e-6
output_voltage = 1.3
vin_min = 0.7
vin_max = 0.9
vdd_min = 1.45
vdd_max = 1.55
vds_headroom = 0.2

In [17]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buffer specification
# note that the -3dB bandwidth of the voltage buffer with gain Av=1 is equal to the unity gain bandwidth
# of the ota, hence we wet them equal here
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loading
# we also add an additional factor of 2 to get more dc gain (and there is power still in the budget)
gm_m12 = f_bw * 3 * 4*np.pi*c_load * 3

print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

gm12 = 0.1131 mS


In [18]:
# since we know gm12 and the gmid we can calculate the bias current
id_m12 = gm_m12 / gm_id_m12
i_total = 2*id_m12
print('i_total (exact) =', round(i_total/1e-6, 1), 'µA')
# we round to 0.5µA bias currents
i_total = max(round(i_total / 1e-6 * 2) / 2 * 1e-6, 0.5e-6)
# here is a manual override to set the current; we keep a reserve of 2µA for bias branch
#i_total = 8e-6
id_m12 = i_total/2

print('i_total (rounded) =', i_total/1e-6, 'µA')
if i_total < i_total_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!') 

i_total (exact) = 16.2 µA
i_total (rounded) = 16.0 µA
[info] power consumption target is met!


In [19]:
# we calculate the dc gain
gm_gds_m12 = lv_pmos.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom, VSB=2*vds_headroom)
gm_gds_m34 = lv_pmos.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=vds_headroom, VSB=0)
gm_gds_m56 = lv_nmos.lookup('GM_GDS', GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom, VSB=2*vds_headroom)
gm_gds_m78 = lv_pmos.lookup('GM_GDS', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom, VSB=vds_headroom)
gm_gds_m90 = lv_nmos.lookup('GM_GDS', GM_ID=gm_id_m90, L=l_90, VDS=vds_headroom, VSB=0)
# conductance of lower cascoded differential pair
gds_m12 = gm_m12 / gm_gds_m12
gds_m56 = gds_m12 / gm_gds_m56
# conductance of upper cascoded current mirror
gm_m34 = gm_id_m34 * i_total/2
gds_m34 = gm_m34 / gm_gds_m34
gds_m78 = gds_m34 / gm_gds_m78

print('gds_12 =', round(gds_m12/1e-6, 3), 'µs')
print('gm_56/gds_56 =',round(float(gm_gds_m56), 1))
print('gds_34 =', round(gds_m34/1e-6, 3), 'µs')
print('gm_78/gds_78 =', round(float(gm_gds_m78), 1))

a0 = gm_m12 / (gds_m56 + gds_m78)
print('a0 =', round(20*np.log10(a0), 1), 'dB')

gds_12 = 2.165 µs
gm_56/gds_56 = 13.3
gds_34 = 2.571 µs
gm_78/gds_78 = 21.7
a0 = 52.1 dB


In [20]:
# we calculate the MOSFET capacitance which adds to Cload, to see the impact on the BW
gm_cgs_m12 = lv_pmos.lookup('GM_CGS', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom, VSB=2*vds_headroom)
gm_cdd_m56 = lv_nmos.lookup('GM_CDD', GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom, VSB=2*vds_headroom)
gm_cdd_m78 = lv_pmos.lookup('GM_CDD', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom, VSB=vds_headroom)


c_load_parasitic = abs(gm_m12/gm_cgs_m12) + abs(gm_m12/gm_cdd_m56) + abs(gm_m34/gm_cdd_m78)
print('additional load capacitance =', round(c_load_parasitic/1e-15, 1), 'fF')

f_bw = gm_m12 / (4*np.pi * (c_load + c_load_parasitic))
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')

additional load capacitance = 121.6 fF
unity gain bandwidth incl. parasitics = 8.02 MHz


In [21]:
# we can now look up the VGS of the MOSFET
vgs_m12 = lv_pmos.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom, VSB=2*vds_headroom)
vgs_m56 = lv_nmos.look_upVGS(GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom, VSB=2*vds_headroom)
vgs_m34 = lv_pmos.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=vds_headroom, VSB=0.0) 
vgs_m78 = lv_pmos.look_upVGS(GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom, VSB=vds_headroom) 

vgs_m90 = lv_nmos.look_upVGS(GM_ID=gm_id_m90, L=l_90, VDS=vds_headroom, VSB=0) 

vgs_mt = lv_pmos.look_upVGS(GM_ID=gm_id_m56, L=l_t, VDS=vds_headroom, VSB=0.0) 

print('vgs_12  =', round(float(vgs_m12), 3), 'V')
print('vgs_56 =', round(float(vgs_m56), 3), 'V')
print('vgs_34  =', round(float(vgs_m34), 3), 'V')
print('vgs_78 =', round(float(vgs_m78), 3), 'V')

print('vgs_90 =', round(float(vgs_m90), 3), 'V')

print('vgs_t  =', round(float(vgs_mt), 3), 'V')

vgs_12  = 0.515 V
vgs_56 = 0.452 V
vgs_34  = 0.479 V
vgs_78 = 0.553 V
vgs_90 = 0.405 V
vgs_t  = 0.512 V


In [22]:
# calculate settling time due to slewing with the calculated bias current
t_slew = (c_load + c_load_parasitic) * output_voltage / i_total
print('slewing time  =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw)
print('settling time =', round(t_settle/1e-6, 3), 'µs')


slewing time  = 0.091 µs
settling time = 0.099 µs


In [23]:
# calculate voltage gain error
gain_error = a0 / (1 + a0)
print('voltage gain error =', round((gain_error-1)*100, 1), '%')


voltage gain error = -0.2 %


In [24]:
# calculate total rms output noise
sth_m12 = lv_nmos.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=vds_headroom, VSB=2*vds_headroom) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)

sth_m34 = lv_nmos.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=vds_headroom, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)

output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34 * gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')

output noise = 110.0 µVrms


In [25]:
# calculate all widths
id_w_m12 = lv_pmos.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom, VSB=2*vds_headroom)
w_12 = id_m12 / id_w_m12
w_12_round = max(round(w_12*2)/2, 0.5)
print('M1/2  W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')

id_m56 = id_m12
id_w_m56 = lv_nmos.lookup('ID_W', GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom, VSB=2*vds_headroom)
w_56 = id_m56 / id_w_m56
w_56_round = max(round(w_56*2)/2, 0.5)
print('M5/6 W =', round(w_56, 2), 'um, rounded W =', w_56_round, 'um')

id_m34 = id_m12
id_w_m34 = lv_pmos.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vds_headroom, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = max(round(w_34*2)/2, 0.5) 
print('M3/4  W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

id_m78 = id_m12
id_w_m78 = lv_pmos.lookup('ID_W', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom, VSB=vds_headroom)
w_78 = id_m78 / id_w_m78
w_78_round = max(round(w_78*2)/2, 0.5) 
print('M7/8 W =', round(w_78, 2), 'um, rounded W =', w_78_round, 'um')

id_m90 = id_m12*2
id_w_m90 = lv_nmos.lookup('ID_W', GM_ID=gm_id_m90, L=l_90, VDS=vds_headroom, VSB=0)
w_90 = id_m90 / id_w_m90
w_90_round = max(round(w_90*2)/2, 0.5) 
print('M9/0 W =', round(w_90, 2), 'um, rounded W =', w_90_round, 'um')










id_w_mt = lv_pmos.lookup('ID_W', GM_ID=gm_id_mt, L=l_t, VDS=vds_headroom, VSB=0)
w_t = i_total / id_w_mt
w_t_round = max(round(w_t*2)/2, 0.5)
print('Mt    W =', round(w_t, 2), 'um, rounded W =', w_t_round, 'um')

w_b = w_t_round * i_bias_in / i_total
print('Mb    W =', round(w_b, 2), 'um')

M1/2  W = 19.47 um, rounded W = 19.5 um
M5/6 W = 2.11 um, rounded W = 2.0 um
M3/4  W = 12.59 um, rounded W = 12.5 um
M7/8 W = 7.94 um, rounded W = 8.0 um
M9/0 W = 4.23 um, rounded W = 4.0 um
Mt    W = 32.59 um, rounded W = 32.5 um
Mb    W = 28.44 um


In [26]:
# Print out final design values
print('Improved OTA dimensioning:')
print('--------------------------')
print('M1/2  W=', w_12_round, ', L=', l_12)
print('M5/6 W=', w_56_round, ', L=', l_56)
print('M3/4  W=', w_34_round, ', L=', l_34)
print('M7/8 W=', w_78_round, ', L=', l_78)
print('M9/0 W=', w_90_round, ', L=', l_90)
print('Mt   W=', w_t_round, ', L=', l_t)
print('Mb   W=', round(w_b, 2), ', L=', l_t)
print()
print('Improved OTA performance summary:')
print('---------------------------------')
print('supply current =', round(i_total/1e-6, 1), 'µA')
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')
print('voltage gain error =', round((gain_error-1)*100, 1), '%')
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')
print('turn-on time (slewing+settling) =', round((t_slew+t_settle)/1e-6, 3), 'µs')
print()
print('Improved OTA bias point check:')
print('------------------------------')
print('headroom M1+M5 =', round(vdd_min-vgs_m34+vgs_m12-vin_max, 3), 'V')
print('headroom M4+M8 =', round(vdd_min-vin_max, 3), 'V')
print('headroom Mt     =', round(vin_min-vgs_m12, 3), 'V')

Improved OTA dimensioning:
--------------------------
M1/2  W= 19.5 , L= 2
M5/6 W= 2.0 , L= 2
M3/4  W= 12.5 , L= 2
M7/8 W= 8.0 , L= 2
M9/0 W= 4.0 , L= 2
Mt   W= 32.5 , L= 5
Mb   W= 28.44 , L= 5

Improved OTA performance summary:
---------------------------------
supply current = 16.0 µA
output noise = 110.0 µVrms
voltage gain error = -0.2 %
unity gain bandwidth incl. parasitics = 8.02 MHz
turn-on time (slewing+settling) = 0.19 µs

Improved OTA bias point check:
------------------------------
headroom M1+M5 = 0.586 V
headroom M4+M8 = 0.55 V
headroom Mt     = 0.185 V
